In [57]:
import geopandas as gpd
import rioxarray
import rasterio
import pandas as pd
from rasterio.features import rasterize
import numpy as np

In [60]:
# Load the GeoPackage using geopandas
gdf = gpd.read_file('NDVI_Jabar_11_24.gpkg')
gdf.head(2)

,ID,Row Labels,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,...,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,geometry
0,ID_1,ID_1,"5895,940564","6904,740395","6691,785125","6464,291358","7581,51905","7842,831603","7055,082617","7119,492888",...,"7665,406967","7471,938528","7155,671143","7948,231461",,,,,,"POLYGON ((108.47500 -7.77500, 108.52500 -7.775..."
1,ID_2,ID_2,"6515,010165","7964,20091","8067,062727","7939,992891","8212,291958","8229,018319","6826,819416","7274,492998",...,"7849,246525","7986,744439","8257,354075","8282,193377",,,,,,"POLYGON ((108.42500 -7.77500, 108.47500 -7.775..."


In [61]:
U=gdf.columns.tolist()
for i in ['ID','Row Labels','2024-08','2024-09','2024-10','2024-11','2024-12']:
    U.remove(i)

In [62]:
# Ensure the attribute columns are numeric
def convert(x):
    try:
        return float(x.strip().replace(',','.'))
    except:
        return np.nan
for index, rows in gdf.iterrows():
    for attr in U:
        if attr!='geometry':
            gdf.at[index,attr]=convert(rows[attr])

In [67]:
# Open the reference TIFF with rioxarray to get its properties
reference_raster = rioxarray.open_rasterio('Curah_Hujan.tif')

# Get the spatial properties from the reference raster
transform = reference_raster.rio.transform()
width, height = reference_raster.rio.width, reference_raster.rio.height
crs = reference_raster.rio.crs

# Prepare the shapes for rasterization
raster_bands = []
for attr in U:
    if attr!='geometry':
        shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf[attr])]
        raster = rasterize(shapes, out_shape=(height, width), transform=transform)
        raster_bands.append(raster)

# Create a new multi-band raster dataset
with rasterio.open(
    'NDVI_11_24_full.tif',
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=len(U)-1,
    dtype=rasterio.float32,
    crs=crs,
    transform=transform,
) as dst:
    for idx, band in enumerate(raster_bands, start=1):
        dst.write(band.astype(rasterio.float32), idx)